In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_mnist
import matplotlib.pyplot as plt
from art.attacks.extraction.copycat_cnn import CopycatCNN
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion.carlini import CarliniL2Method
from art.utils import random_targets
from art.defences.trainer.adversarial_trainer import AdversarialTrainer
from art.estimators.certification.randomized_smoothing import TensorFlowV2RandomizedSmoothing

In [ ]:
(X_train, y_train), (X_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

nb_classes = 10

In [ ]:
model = Sequential()
model.add(Conv2D(1 , kernel_size=(7, 7), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten())
model.add(Dense(nb_classes, activation='softmax'))
model.compile()

model.summary()

In [ ]:
nb_epochs = 5
batch_size = 128
input_shape = X_train.shape[1:]
alpha = 0.001
sample_size = 100

In [ ]:
optimizer = Adam(learning_rate=alpha)
loss_object = CategoricalCrossentropy(from_logits=False)
def train_step(model, images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))


### RandomizedSmoothing

In [ ]:
sigmas = {
    'Smoothed Classifier, sigma=0.1': 0.1,
    'Smoothed Classifier, sigma=0.25': 0.25,
    'Smoothed Classifier, sigma=0.5': 0.5,
}
classifiers = {}

In [ ]:
def get_cert_acc(radius, pred, y_test):
    
    rad_list = np.linspace(0, 2.25, 201)
    cert_acc = []
    num_cert = len(np.where(radius > 0)[0])

    for r in rad_list:
        rad_idx = np.where(radius > r)[0]
        y_test_subset = y_test[rad_idx]
        cert_acc.append(np.sum(pred[rad_idx] == np.argmax(y_test_subset, axis=1)) / num_cert)

    return cert_acc

In [ ]:
for name in sigmas:
    print(name)
    sigma = sigmas[name]

    classifier = TensorFlowV2RandomizedSmoothing(
        model=model,
        nb_classes=nb_classes,
        input_shape=input_shape,
        loss_object=loss_object,
        train_step=train_step,
        channels_first=False,
        clip_values=(min_pixel_value, max_pixel_value),
        sample_size=sample_size,
        scale=sigma,
        alpha=alpha)
    
    print('Training')

    classifier.fit(X_train, y_train, nb_epochs=nb_epochs, batch_size=batch_size)

    print('Certifying')

    cert_preds, radius = classifier.certify(X_test, n=10)

    rad_list = np.linspace(0, 2.25, 201)

    plt.plot(rad_list, get_cert_acc(radius, cert_preds, y_test), label=name)

    classifiers[name] = classifier


plt.xlabel('Radius')
plt.ylabel('Certified Accuracy')
plt.legend()
plt.show()

In [ ]:
for classifier in classifiers:
    preds = classifiers[classifier].predict(X_test)
    acc = np.sum(np.argmax(preds, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print(f'Accuracy of {classifier}: {100 * acc} %')

### Resistance to FGSM(Fast Gradient Sign Method)

In [ ]:
fgm_attack = FastGradientMethod(estimator=classifiers['Smoothed Classifier, sigma=0.1'], eps=0.1)

X_test_adv = fgm_attack.generate(x=X_test)

for classifier in classifiers:
    preds = classifiers[classifier].predict(X_test_adv)
    acc = np.sum(np.argmax(preds, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print(f'Accuracy of {classifier} on adversarial test examples: {100 * acc} %')
